# Demo: Generation & extraction of the IPEs for the HF Modules

Generate & extract relevant IPEs for the HF modules.

__Pre-requisites:__

- `.env` with your AWS credentials

## Modules/Libraries

In [1]:
from icefabric.modules import *
from pyiceberg.catalog import load_catalog
from icefabric.schemas.hydrofabric import HydrofabricDomains

## User Arguments

In [2]:
catalog = load_catalog("glue",
                       **{"type":"glue", "glue.region":"us-east-1"})
domain = HydrofabricDomains.CONUS
identifier = '06710385'


## TopModel

In [3]:
topmodel_pydantic_models = get_topmodel_parameters(catalog,
                                                   domain,
                                                   identifier)
topmodel_pydantic_models

Tracing upstream from outlet(s): ['nex-1571937']
Found 36 topology connections
Found 37 upstream segments


[Topmodel(catchment='cat-1571931', divide_id='cat-1571931', num_sub_catchments=1, imap=1, yes_print_output=1, num_topodex_values=4, area=1, num_channels=1, cum_dist_area_with_dist=1.0, dist_from_outlet=5620.0, szm=0.0125, t0=7.5e-05, td=20, chv=1000, rv=1000, srmax=0.04, Q0=3.28e-05, sr0=0, infex=0, xk0=2, hf=0.1, dth=0.1),
 Topmodel(catchment='cat-1571935', divide_id='cat-1571935', num_sub_catchments=1, imap=1, yes_print_output=1, num_topodex_values=4, area=1, num_channels=1, cum_dist_area_with_dist=1.0, dist_from_outlet=3637.0, szm=0.0125, t0=7.5e-05, td=20, chv=1000, rv=1000, srmax=0.04, Q0=3.28e-05, sr0=0, infex=0, xk0=2, hf=0.1, dth=0.1),
 Topmodel(catchment='cat-1571936', divide_id='cat-1571936', num_sub_catchments=1, imap=1, yes_print_output=1, num_topodex_values=4, area=1, num_channels=1, cum_dist_area_with_dist=1.0, dist_from_outlet=7414.0, szm=0.0125, t0=7.5e-05, td=20, chv=1000, rv=1000, srmax=0.04, Q0=3.28e-05, sr0=0, infex=0, xk0=2, hf=0.1, dth=0.1),
 Topmodel(catchment='c

## OWP NOAH MODULAR

In [4]:
noahowp_pydantic_models = get_noahowp_parameters(catalog,
                                                 domain,
                                                 identifier)
noahowp_pydantic_models

Tracing upstream from outlet(s): ['nex-1571937']
Found 36 topology connections
Found 37 upstream segments


[NoahOwpModular(catchment='cat-1571931', dt=3600.0, startdate='202408260000', enddate='202408260000', forcing_filename='.', output_filename='.', parameter_dir='test', general_table='GENPARM.TBL', soil_table='SOILPARM.TBL', noahowp_table='MPTABLE.TBL', soil_class_name='STAS', veg_class_name='USGS', lat=1883189.9997, lon=-812875.5000270001, terrain_slope=85.94976013129424, azimuth=109.38325027089329, ZREF=10.0, rain_snow_thresh=0.5, precip_phase_option=6, snow_albedo_option=1, dynamic_veg_option=4, runoff_option=3, drainage_option=8, frozen_soil_option=1, dynamic_vic_option=1, radiative_transfer_option=3, sfc_drag_coeff_option=1, canopy_stom_resist_option=1, crop_model_option=0, snowsoil_temp_time_option=3, soil_temp_boundary_option=2, supercooled_water_option=1, stomatal_resistance_option=1, evap_srfc_resistance_option=4, subsurface_option=2, isltyp=3.0, nsoil=4, nsnow=3, nveg=27, vegtyp=14, croptype=0, sfctyp=1, soilcolor=4, dzsnso=[0.0, 0.0, 0.0, 0.1, 0.3, 0.6, 1.0], sice=[0.0, 0.0, 0

## LASAM

In [5]:
lasam_pydantic_models = get_lasam_parameters(catalog,
                                             domain,
                                             identifier,
                                             sft_included=True,
                                             soil_params_file="vG_default_params_HYDRUS.dat")
lasam_pydantic_models

Tracing upstream from outlet(s): ['nex-1571937']
Found 36 topology connections
Found 37 upstream segments


[LASAM(catchment='cat-1571931', verbosity='none', soil_params_file='vG_default_params_HYDRUS.dat', layer_thickness='200.0[cm]', initial_psi='2000.0[cm]', timestep='300[sec]', endtime='1000[hr]', forcing_resolution='3600[sec]', ponded_depth_max='1.1[cm]', use_closed_form_G=False, layer_soil_type=3.0, max_soil_types=15, wilting_point_psi='15495.0[cm]', field_capacity_psi='340.9[cm]', giuh_ordinates=[0.06, 0.51, 0.28, 0.12, 0.03], calib_params=True, adaptive_timestep=True, sft_coupled=True, soil_z=[10, 30, 100.0, 200.0]),
 LASAM(catchment='cat-1571935', verbosity='none', soil_params_file='vG_default_params_HYDRUS.dat', layer_thickness='200.0[cm]', initial_psi='2000.0[cm]', timestep='300[sec]', endtime='1000[hr]', forcing_resolution='3600[sec]', ponded_depth_max='1.1[cm]', use_closed_form_G=False, layer_soil_type=3.0, max_soil_types=15, wilting_point_psi='15495.0[cm]', field_capacity_psi='340.9[cm]', giuh_ordinates=[0.06, 0.51, 0.28, 0.12, 0.03], calib_params=True, adaptive_timestep=True, 

In [6]:
lasam_pydantic_models = get_lasam_parameters(catalog,
                                             domain,
                                             identifier,
                                             sft_included=False,
                                             soil_params_file="vG_default_params_HYDRUS.dat")
lasam_pydantic_models

Tracing upstream from outlet(s): ['nex-1571937']
Found 36 topology connections
Found 37 upstream segments


[LASAM(catchment='cat-1571931', verbosity='none', soil_params_file='vG_default_params_HYDRUS.dat', layer_thickness='200.0[cm]', initial_psi='2000.0[cm]', timestep='300[sec]', endtime='1000[hr]', forcing_resolution='3600[sec]', ponded_depth_max='1.1[cm]', use_closed_form_G=False, layer_soil_type=3.0, max_soil_types=15, wilting_point_psi='15495.0[cm]', field_capacity_psi='340.9[cm]', giuh_ordinates=[0.06, 0.51, 0.28, 0.12, 0.03], calib_params=True, adaptive_timestep=True, sft_coupled=False, soil_z=[10, 30, 100.0, 200.0]),
 LASAM(catchment='cat-1571935', verbosity='none', soil_params_file='vG_default_params_HYDRUS.dat', layer_thickness='200.0[cm]', initial_psi='2000.0[cm]', timestep='300[sec]', endtime='1000[hr]', forcing_resolution='3600[sec]', ponded_depth_max='1.1[cm]', use_closed_form_G=False, layer_soil_type=3.0, max_soil_types=15, wilting_point_psi='15495.0[cm]', field_capacity_psi='340.9[cm]', giuh_ordinates=[0.06, 0.51, 0.28, 0.12, 0.03], calib_params=True, adaptive_timestep=True,

## LSTM

In [7]:
lstm_pydantic_models = get_lstm_parameters(catalog,
                                           domain,
                                           identifier)
lstm_pydantic_models

Tracing upstream from outlet(s): ['nex-1571937']
Found 36 topology connections
Found 37 upstream segments


[LSTM(catchment='cat-1571931', area_sqkm=3.7552500359994037, basin_id='06710385', basin_name='', elev_mean=303202.5035878757, inital_state='zero', lat=1883189.9997, lon=-812875.5000270001, slope_mean=85.94976013129424, timestep='1 hour', train_cfg_file='', verbose='0'),
 LSTM(catchment='cat-1571935', area_sqkm=9.163799554500065, basin_id='06710385', basin_name='', elev_mean=242323.2400662253, inital_state='zero', lat=1884884.99985, lon=-798566.2496718751, slope_mean=86.50193995060438, timestep='1 hour', train_cfg_file='', verbose='0'),
 LSTM(catchment='cat-1571936', area_sqkm=10.91474954100026, basin_id='06710385', basin_name='', elev_mean=231378.65634606397, inital_state='zero', lat=1886130.0, lon=-795321.7499846249, slope_mean=86.57582290477289, timestep='1 hour', train_cfg_file='', verbose='0'),
 LSTM(catchment='cat-1571975', area_sqkm=8.782650049500255, basin_id='06710385', basin_name='', elev_mean=355076.12212954345, inital_state='zero', lat=1880910.0, lon=-815677.4996999999, slop

## SMP

In [8]:
module = None #'CFE-S', 'CFE-X', 'LASAM', 'TopModel'
smp_pydantic_models = get_smp_parameters(catalog,
                                         domain,
                                         identifier,
                                         module)
smp_pydantic_models

Tracing upstream from outlet(s): ['nex-1571937']
Found 36 topology connections
Found 37 upstream segments


[SMP(catchment='cat-1571931', verbosity='none', soil_params_smcmax=0.48807379603385925, soil_params_b=5.021543025970459, soil_params_satpsi=0.010489353695051074, soil_z=[0.1, 0.3, 1.0, 2.0], soil_moisture_fraction_depth=0.4, soil_storage_model='NA', soil_storage_depth='NA', water_table_based_method='NA', soil_moisture_profile_option='NA', soil_depth_layers='NA', water_table_depth='NA'),
 SMP(catchment='cat-1571935', verbosity='none', soil_params_smcmax=0.48807379603385925, soil_params_b=5.021543025970459, soil_params_satpsi=0.029248272386826733, soil_z=[0.1, 0.3, 1.0, 2.0], soil_moisture_fraction_depth=0.4, soil_storage_model='NA', soil_storage_depth='NA', water_table_based_method='NA', soil_moisture_profile_option='NA', soil_depth_layers='NA', water_table_depth='NA'),
 SMP(catchment='cat-1571936', verbosity='none', soil_params_smcmax=0.48568305373191833, soil_params_b=5.021543025970459, soil_params_satpsi=0.02200991659940715, soil_z=[0.1, 0.3, 1.0, 2.0], soil_moisture_fraction_depth=0

## Snow17

In [9]:
snow17_pydantic_models = get_snow17_parameters(catalog,
                                               domain,
                                               identifier,
                                               conus_only=True)

snow17_pydantic_models

Tracing upstream from outlet(s): ['nex-1571937']
Found 36 topology connections
Found 37 upstream segments


[Snow17(catchment='cat-1571931', hru_id='cat-1571931', hru_area=3.7552500359994037, latitude=1883189.9997, elev=303202.5035878757, scf=1.1, mf_max=0.740213811397552, mf_min=0.237466230988503, uadj=0.0136389946565032, si=500.0, pxtemp=1.0, nmf=0.15, tipm=0.1, mbase=0.0, plwhc=0.03, daygm=0.0, adc1=0.05, adc2=0.1, adc3=0.2, adc4=0.3, adc5=0.4, adc6=0.5, adc7=0.6, adc8=0.7, adc9=0.8, adc10=0.9, adc11=1.0),
 Snow17(catchment='cat-1571935', hru_id='cat-1571935', hru_area=9.163799554500065, latitude=1884884.99985, elev=242323.2400662253, scf=1.1, mf_max=0.834751725196838, mf_min=0.223617702722549, uadj=0.016959622502327, si=500.0, pxtemp=1.0, nmf=0.15, tipm=0.1, mbase=0.0, plwhc=0.03, daygm=0.0, adc1=0.05, adc2=0.1, adc3=0.2, adc4=0.3, adc5=0.4, adc6=0.5, adc7=0.6, adc8=0.7, adc9=0.8, adc10=0.9, adc11=1.0),
 Snow17(catchment='cat-1571936', hru_id='cat-1571936', hru_area=10.91474954100026, latitude=1886130.0, elev=231378.65634606397, scf=1.1, mf_max=0.985058844089508, mf_min=0.276576101779938

In [10]:
snow17_pydantic_models = get_snow17_parameters(catalog,
                                               domain,
                                               identifier,
                                               conus_only=False)
snow17_pydantic_models

Tracing upstream from outlet(s): ['nex-1571937']
Found 36 topology connections
Found 37 upstream segments


[Snow17(catchment='cat-1571931', hru_id='cat-1571931', hru_area=3.7552500359994037, latitude=1883189.9997, elev=303202.5035878757, scf=1.1, mf_max=1.0, mf_min=0.2, uadj=0.05, si=500.0, pxtemp=1.0, nmf=0.15, tipm=0.1, mbase=0.0, plwhc=0.03, daygm=0.0, adc1=0.05, adc2=0.1, adc3=0.2, adc4=0.3, adc5=0.4, adc6=0.5, adc7=0.6, adc8=0.7, adc9=0.8, adc10=0.9, adc11=1.0),
 Snow17(catchment='cat-1571935', hru_id='cat-1571935', hru_area=9.163799554500065, latitude=1884884.99985, elev=242323.2400662253, scf=1.1, mf_max=1.0, mf_min=0.2, uadj=0.05, si=500.0, pxtemp=1.0, nmf=0.15, tipm=0.1, mbase=0.0, plwhc=0.03, daygm=0.0, adc1=0.05, adc2=0.1, adc3=0.2, adc4=0.3, adc5=0.4, adc6=0.5, adc7=0.6, adc8=0.7, adc9=0.8, adc10=0.9, adc11=1.0),
 Snow17(catchment='cat-1571936', hru_id='cat-1571936', hru_area=10.91474954100026, latitude=1886130.0, elev=231378.65634606397, scf=1.1, mf_max=1.0, mf_min=0.2, uadj=0.05, si=500.0, pxtemp=1.0, nmf=0.15, tipm=0.1, mbase=0.0, plwhc=0.03, daygm=0.0, adc1=0.05, adc2=0.1, a

## SAC SMA

In [11]:
sacsma_pydantic_models = get_sacsma_parameters(catalog,
                                               domain,
                                               identifier,
                                               conus_only=True)
sacsma_pydantic_models

Tracing upstream from outlet(s): ['nex-1571937']
Found 36 topology connections
Found 37 upstream segments


[SacSma(catchment='cat-1571931', hru_id='cat-1571931', hru_area=3.7552500359994037, uztwm=57.7215309143066, uzfwm=40.0665435791016, lztwm=175.447631835938, lzfpm=97.7323684692383, lzfsm=14.0631313323975, adimp=0.0, uzk=0.503200888633728, lzpk=0.0162544026970863, lzsk=0.180965483188629, zperc=190.605392456055, rexp=1.70314168930054, pctim=0.0, pfree=0.113952964544296, riva=0.0, side=0.0, rserv=0.3),
 SacSma(catchment='cat-1571935', hru_id='cat-1571935', hru_area=9.163799554500065, uztwm=35.7420539855957, uzfwm=35.3901557922363, lztwm=63.5002555847168, lzfpm=54.8282012939453, lzfsm=6.44366645812988, adimp=0.0, uzk=0.547945439815521, lzpk=0.0105827422812581, lzsk=0.194084122776985, zperc=78.3233795166016, rexp=1.51577663421631, pctim=0.0, pfree=0.0940396338701248, riva=0.0, side=0.0, rserv=0.3),
 SacSma(catchment='cat-1571936', hru_id='cat-1571936', hru_area=10.91474954100026, uztwm=39.6380348205566, uzfwm=36.4422798156738, lztwm=69.1735534667969, lzfpm=56.2926445007324, lzfsm=7.365390300

In [12]:
sacsma_pydantic_models = get_sacsma_parameters(catalog,
                                               domain,
                                               identifier,
                                               conus_only=False)
sacsma_pydantic_models

Tracing upstream from outlet(s): ['nex-1571937']
Found 36 topology connections
Found 37 upstream segments


[SacSma(catchment='cat-1571931', hru_id='cat-1571931', hru_area=3.7552500359994037, uztwm=75.0, uzfwm=30.0, lztwm=150.0, lzfpm=300.0, lzfsm=150.0, adimp=0.0, uzk=0.3, lzpk=0.01, lzsk=0.1, zperc=100.0, rexp=2.0, pctim=0.0, pfree=0.1, riva=0.0, side=0.0, rserv=0.3),
 SacSma(catchment='cat-1571935', hru_id='cat-1571935', hru_area=9.163799554500065, uztwm=75.0, uzfwm=30.0, lztwm=150.0, lzfpm=300.0, lzfsm=150.0, adimp=0.0, uzk=0.3, lzpk=0.01, lzsk=0.1, zperc=100.0, rexp=2.0, pctim=0.0, pfree=0.1, riva=0.0, side=0.0, rserv=0.3),
 SacSma(catchment='cat-1571936', hru_id='cat-1571936', hru_area=10.91474954100026, uztwm=75.0, uzfwm=30.0, lztwm=150.0, lzfpm=300.0, lzfsm=150.0, adimp=0.0, uzk=0.3, lzpk=0.01, lzsk=0.1, zperc=100.0, rexp=2.0, pctim=0.0, pfree=0.1, riva=0.0, side=0.0, rserv=0.3),
 SacSma(catchment='cat-1571975', hru_id='cat-1571975', hru_area=8.782650049500255, uztwm=75.0, uzfwm=30.0, lztwm=150.0, lzfpm=300.0, lzfsm=150.0, adimp=0.0, uzk=0.3, lzpk=0.01, lzsk=0.1, zperc=100.0, rexp=2.

## T-Route

In [13]:
troute_pydantic_models = get_troute_parameters(catalog,
                                               domain,
                                               identifier)
troute_pydantic_models

Tracing upstream from outlet(s): ['nex-1571937']
Found 36 topology connections
Found 37 upstream segments


[TRoute(catchment='cat-1571931', bmi_param={'flowpath_columns': ['id', 'toid', 'lengthkm'], 'attributes_columns': ['attributes_id', 'rl_gages', 'rl_NHDWaterbodyComID', 'MusK', 'MusX', 'n', 'So', 'ChSlp', 'BtmWdth', 'nCC', 'TopWdthCC', 'TopWdth'], 'waterbody_columns': ['hl_link', 'ifd', 'LkArea', 'LkMxE', 'OrificeA', 'OrificeC', 'OrificeE', 'WeirC', 'WeirE', 'WeirL'], 'network_columns': ['network_id', 'hydroseq', 'hl_uri']}, log_param={'showtiming': True, 'log_level': 'DEBUG'}, nwtopo_param={'supernetwork_parameters': {'geo_file_path': 'gauge_06710385.gpkg'}, 'waterbody_parameters': {'level_pool': {'level_pool_waterbody_parameter_file_path': 'gauge_06710385.gpkg'}}}, comp_param={'parallel_compute_method': 'by-subnetwork-jit-clustered', 'subnetwork_target_size': 10000, 'cpu_pool': 16, 'compute_kernel': 'V02-structured', 'assume_short_ts': True, 'restart_parameters': {'start_datetime': ''}, 'forcing_parameters': {'qts_subdivisions': 12, 'dt': 300, 'qlat_input_folder': '.', 'qlat_file_patt